In [1]:
#! /usr/bin/python3.5

In [2]:
a=["A","B","C","D","hh"]

In [3]:
def chunkIt(seq, num):
   
    avg = len(seq) / float(num)
    out = []
    last = 0.0
    if num > 1:
        while last < len(seq):
            out.append(seq[int(last):int(last + avg)])
            last += avg
    else:
        return seq

    return out

In [4]:
def model(a_list,i):
    print("list model",a_list,i)

In [5]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
    #return [x.name for x in local_device_protos]

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [6]:
import subprocess, re, os, sys #https://github.com/yaroslavvb/stuff/blob/master/notebook_util.py
def run_command(cmd):
    """Run command, return output as string."""
    
    output = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True).communicate()[0]
    return output.decode("ascii")


In [7]:
def list_available_gpus():
    """Returns list of available GPU ids."""
    
    output = run_command("nvidia-smi -L")
    # lines of the form GPU 0: TITAN X
    gpu_regex = re.compile(r"GPU (?P<gpu_id>\d+):")
    result = []
    for line in output.strip().split("\n"):
        m = gpu_regex.match(line)
        assert m, "Couldnt parse "+line
        result.append(int(m.group("gpu_id")))
    return result

In [8]:
def gpu_memory_map():
    """Returns map of GPU id to memory allocated on that GPU."""

    output = run_command("nvidia-smi")
    print("nvidia-smi",output)
    gpu_output = output[output.find("GPU Memory"):]
    print("GPU Memory",output)
    # lines of the form
    # |    0      8734    C   python                                       11705MiB |
    memory_regex = re.compile(r"[|]\s+?(?P<gpu_id>\d+)\D+?(?P<pid>\d+).+[ ](?P<gpu_memory>\d+)MiB")
    #print("memory_regex",memory_regex)
    rows = gpu_output.split("\n")
    #print("rows",rows)
    result = {gpu_id: 0 for gpu_id in list_available_gpus()}
    #print("result",result)
    for row in gpu_output.split("\n"):
        m = memory_regex.search(row)
        if not m:
            continue
        gpu_id = int(m.group("gpu_id"))
        gpu_memory = int(m.group("gpu_memory"))
        result[gpu_id] += gpu_memory
    return result

In [9]:

import os
import GPUtil as GPU
import tensorflow as tf
#import threading
import timeit
import time
#from tensorflow import logging
from tensorflow.python.client import timeline
#os.environ['TF_CPP_MIN_VLOG_LEVEL']='3'
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#logging.set_verbosity(logging.INFO)
#logging.vlog(1, "Registering %s (%s) in %s.", name, candidate, self._name)

config = tf.ConfigProto(device_count={"GPU": 2,"CPU":1},
                        allow_soft_placement=True,
                        inter_op_parallelism_threads=16,
                        intra_op_parallelism_threads=16,
                        use_per_session_threads=True,
                        log_device_placement=True)
config.gpu_options.allow_growth = True

#run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
#run_metadata = tf.RunMetadata()
print("************Start*************")
print("gpu_memory_map",gpu_memory_map())
GPU.showUtilization(all=True)
print("Start GPU",run_command("nvidia-smi -a"))
#import setGPU
#print("setGPU.stats",setGPU.stats)
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    GPU_LIST=[]
    list_file_gpu=[]
    import os
    #os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"
    with tf.device('/cpu'):
       
        #print("setGPU.bestGPU",setGPU.bestGPU)
        #print("setGPU.ratios",setGPU.ratios)
        #print("setGPU.ids",setGPU.ids)
        
        print("Start Time CPU",time.time(), time.clock())
        start = timeit.default_timer()
        GPU_LIST=get_available_gpus()
        list_file_gpu=chunkIt(a,len(GPU_LIST))
        print(list_file_gpu)
        for i in range(len(GPU_LIST)):
            with tf.device(tf.DeviceSpec(device_type="GPU", device_index=i)):
                start_time=time.time()
                start_oclock=time.clock()
                print("Start Time GPU",i,"Time is ",start_time,start_oclock)
                #gpu_full_process(file_names,path_data_base,path_tf,path_non_redundant,path_idf,file_path_sim)
                #Your statements here
                print("In GPU Dev",i)
                model(list_file_gpu[i],i)
                end_time=time.time()
                end_oclock=time.clock()
                print("End Time GPU",i,"Time is ",end_time,end_oclock)
                total_time_GPU=end_time-start_time
                total_clock_GPU=end_oclock-start_oclock
                print("total_time_GPU",total_time_GPU)
                print("total_clock_GPU",total_clock_GPU)
        
        print("************End*************")
        print("setGPU.stats",setGPU.stats)
        #print("setGPU.bestGPU",setGPU.bestGPU)
        #print("setGPU.ratios",setGPU.ratios)
        #print("setGPU.ids",setGPU.ids)
        #print("gpu_memory_map",gpu_memory_map())
        stop = timeit.default_timer()
        print("End Time",time.time(), time.clock())
        #GPU.showUtilization(all=True)#https://github.com/anderskm/gputil
        print("****************End***************")
        print("gpu_memory_map",gpu_memory_map())
        GPU.showUtilization(all=True)
        print("End GPU",run_command("nvidia-smi -a"))
        #tl = timeline.Timeline(run_metadata.step_stats)
        #print("tl.generate_chrome_trace_format",tl.generate_chrome_trace_format(show_memory=True))

************Start*************
Start GPU 
==============NVSMI LOG==============

Timestamp                           : Thu Dec  7 12:40:45 2017
Driver Version                      : 384.90

Attached GPUs                       : 1
GPU 00000000:01:00.0
    Product Name                    : GeForce GT 540M
    Product Brand                   : GeForce
    Display Mode                    : N/A
    Display Active                  : N/A
    Persistence Mode                : Disabled
    Accounting Mode                 : N/A
    Accounting Mode Buffer Size     : N/A
    Driver Model
        Current                     : N/A
        Pending                     : N/A
    Serial Number                   : N/A
    GPU UUID                        : GPU-687ba13e-aaa5-174c-8be0-a34f9a2e7013
    Minor Number                    : 0
    VBIOS Version                   : 70.08.4A.00.22
    MultiGPU Board                  : N/A
    Board ID                        : N/A
    GPU Part Number                

ZeroDivisionError: float division by zero